In [28]:
import pandas as pd
from util.annotator_helper import form_label_dataframes, get_agreement


In [29]:
human_labelled = pd.read_csv(
    'data/acl/sections/labelled/manuel/text_sim_header/processed/partially_agreed_non_irrelevants.csv', index_col=0)
indexes = human_labelled.index
selected_metric = "weighted_kappa"


In [30]:
human_labelled.shape, indexes.shape


((731, 2), (731,))

In [31]:
main_dir = "data/acl/sections/labelled/zeroshot/"

header_labelled_df = pd.read_csv(f"{main_dir}header.csv", index_col=0)
header_concat_labelled_df = pd.read_csv(
    f"{main_dir}header_plus.csv", index_col=0)
content_labelled_df = pd.read_csv(f"{main_dir}content.csv", index_col=0)
header_content_labelled_df = pd.read_csv(
    f"{main_dir}header_content.csv", index_col=0)
header_concat_content_labelled_df = pd.read_csv(
    f"{main_dir}header_plus_content.csv", index_col=0)
grouped_labelled_df = pd.read_csv(f"{main_dir}grouped.csv", index_col=0)


In [32]:
human_labelled = human_labelled.assign(
    header_labelled=header_labelled_df.loc[indexes].label.values, header_labelled_score=header_labelled_df.loc[indexes].score.values)

human_labelled = human_labelled.assign(
    header_labelled=header_labelled_df.loc[indexes].label.values, header_labelled_score=header_labelled_df.loc[indexes].score.values)
human_labelled = human_labelled.assign(
    header_concat_labelled=header_concat_labelled_df.loc[indexes].label.values, header_concat_labelled_score=header_concat_labelled_df.loc[indexes].score.values)
human_labelled = human_labelled.assign(
    content_labelled=content_labelled_df.loc[indexes].label.values, content_labelled_score=content_labelled_df.loc[indexes].score.values)
human_labelled = human_labelled.assign(
    header_content_labelled=header_content_labelled_df.loc[indexes].label.values, header_content_labelled_score=header_content_labelled_df.loc[indexes].score.values)
human_labelled = human_labelled.assign(header_concat_content_labelled=header_concat_content_labelled_df.loc[
                                       indexes].label.values, header_concat_content_labelled_score=header_concat_content_labelled_df.loc[indexes].score.values)


In [33]:
all_scores = pd.DataFrame()
all_scores = all_scores.assign(
    header_labelled=header_labelled_df.label.values, header_labelled_score=header_labelled_df.score.values)
all_scores = all_scores.assign(
    header_concat_labelled=header_concat_labelled_df.label.values, header_concat_labelled_score=header_concat_labelled_df.score.values)
all_scores = all_scores.assign(
    content_labelled=content_labelled_df.label.values, content_labelled_score=content_labelled_df.score.values)
all_scores = all_scores.assign(
    header_content_labelled=header_content_labelled_df.label.values, header_content_labelled_score=header_content_labelled_df.score.values)
all_scores = all_scores.assign(header_concat_content_labelled=header_concat_content_labelled_df.label.values,
                               header_concat_content_labelled_score=header_concat_content_labelled_df.score.values)


In [34]:
print("Similarity score means for each labelling method that agreed with the human labelling".title())
pd.DataFrame(human_labelled.mean(numeric_only=True).round(4).to_dict(), index=["mean"]).T.style.background_gradient(
    cmap='viridis', axis=0).format("{:.3f}")


Similarity Score Means For Each Labelling Method That Agreed With The Human Labelling


,mean
header_labelled_score,0.698
header_concat_labelled_score,0.720
content_labelled_score,0.692
header_content_labelled_score,0.735
header_concat_content_labelled_score,0.754


In [35]:
all_scores_means = {}

for labelling in all_scores.select_dtypes(exclude=['float64']).columns.tolist():
    mean = all_scores[f"{labelling}_score"].mean()
    all_scores_means[f"{labelling}_score"] = round(mean, 3)

print("Similarity score means for each labelling method".title())
all_scores_means["grouped_score"] = grouped_labelled_df.mean(
    numeric_only=True).round(4).to_dict()["score"]
similarity_scores = pd.DataFrame(all_scores_means, index=["mean"]).T
similarity_scores.style.background_gradient(
    cmap='viridis', axis=0).format("{:.3f}")


Similarity Score Means For Each Labelling Method


,mean
header_labelled_score,0.638
header_concat_labelled_score,0.653
content_labelled_score,0.661
header_content_labelled_score,0.686
header_concat_content_labelled_score,0.700
grouped_score,0.678


In [36]:
print("Similarity score means for each labelling method That Agreed With The Human Labelling".title())
labelling_based_scores_means = []
for labelling in human_labelled.iloc[:, 2:].select_dtypes(exclude=['float64']).columns.tolist():
    means = human_labelled[human_labelled.human ==
                           human_labelled[labelling]][["human", labelling, f"{labelling}_score"]].groupby("human").mean(numeric_only=True).round(3).reset_index()
    labelling_based_scores_means.append(means)
labelling_based_scores_mean_all = pd.concat(
    labelling_based_scores_means, axis=1).T.drop_duplicates().T
labelling_based_scores_mean_all.set_index("human", inplace=True)
labelling_based_scores_mean_all.style.highlight_max(
    axis=1, color='green').format("{:.3f}")


Similarity Score Means For Each Labelling Method That Agreed With The Human Labelling


,header_labelled_score,header_concat_labelled_score,content_labelled_score,header_content_labelled_score,header_concat_content_labelled_score
human,,,,,
evaluation,0.838,0.845,0.836,0.878,0.902
introduction,0.711,0.759,0.782,0.840,0.838
pretrained_model,0.916,0.879,0.652,0.722,0.717
requirements,0.753,0.754,0.655,0.764,0.782
results,0.778,0.925,0.817,0.864,0.921
training,0.964,0.963,0.906,0.931,0.937


In [37]:
print("Agreement score for each labelling method with human annotation".title())
agreements = {}
for labelling in human_labelled.iloc[:, 2:].select_dtypes(exclude=['float64']).columns.tolist():
    annotations = form_label_dataframes(
        human_labelled[["human", labelling]])
    agreements[labelling] = get_agreement(annotations, selected_metric)
kappa = pd.DataFrame(agreements, index=["kappa"]).T
kappa.style.background_gradient(
    cmap='viridis', axis=0).format("{:.3f}")


Agreement Score For Each Labelling Method With Human Annotation


,kappa
header_labelled,0.314
header_concat_labelled,0.332
content_labelled,0.236
header_content_labelled,0.338
header_concat_content_labelled,0.341


In [38]:
print("Agreement score means for each labelling method".title())
labelling_based_agreements = {}
for labelling in human_labelled.iloc[:, 2:].select_dtypes(exclude=['float64']).columns.tolist():
    means = {}
    for label in human_labelled.human.unique():
        formed_df = form_label_dataframes(
            human_labelled.loc[human_labelled.human == label][["human", labelling]])
        mean = get_agreement(formed_df, selected_metric)
        means[label] = mean
    labelling_based_agreements[labelling] = means
labelling_based_agreement_mean_all = pd.DataFrame(labelling_based_agreements)
labelling_based_agreement_mean_all.style.highlight_max(
    axis=1, color='green').format("{:.3f}")


Agreement Score Means For Each Labelling Method


,header_labelled,header_concat_labelled,content_labelled,header_content_labelled,header_concat_content_labelled
requirements,0.000,0.000,0.000,0.000,0.000
introduction,0.000,0.000,0.000,0.000,0.000
pretrained_model,0.000,0.000,0.000,0.000,0.000
evaluation,0.000,0.000,0.000,0.000,0.000
training,0.000,0.000,0.000,0.000,0.000
results,0.000,0.000,0.000,0.000,0.000


In [39]:
similarity_scores.index = similarity_scores.index.str.replace("_score", "")
res = similarity_scores.join(kappa)
res.index = res.index.str.replace(
    "_labelled", "").str.replace("_", "+").str.title()


In [40]:
res.rename(columns={"mean": "Classification Score(Avg.)",
           "kappa": "Agreement Score"}, inplace=True)
res


,Classification Score(Avg.),Agreement Score
Header,0.638,0.314198
Header+Concat,0.653,0.331889
Content,0.661,0.236419
Header+Content,0.686,0.338260
Header+Concat+Content,0.700,0.340852
Grouped,0.678,NaN
